
# Query Reformulation using the UniversalDeepTransformer API

This notebook shows how to build a query reformulation model with ThirdAI's Universal Deep Transformer (UDT) model, our all-purpose API for classification tasks on tabular datasets and query reformulation. In this demo, we will train and evaluate the model on a spelling correction dataset.

To run this notebook, you will need to obtain a ThirdAI license at the following link if you have not already: https://www.thirdai.com/try-bolt/

In [2]:
!pip3 install thirdai==0.5.4
!pip3 install pandas
!pip3 install datasets 


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


## Dataset Download

We will use the utils module in this repository to download and pre-process a dataset from HuggingFace. The dataset we will use from HuggingFace is typically used for semantic sentence similarity. We will pre-process it by adding noise so that it is suitable for query reformulation. You can replace this step and the next with a UDT initialization that is specific for your dataset - as long as your input dataset consists of a column with incorrect queries and a column with their target reformulations. 

In [3]:
from utils import prepare_query_reformulation_data
import pandas as pd

train_filename, test_filename, inference_batch = prepare_query_reformulation_data()


Using custom data configuration embedding-data--sentence-compression-05ca3d220d185946
Reusing dataset json (/Users/blaisethirdai/.cache/huggingface/datasets/embedding-data___json/embedding-data--sentence-compression-05ca3d220d185946/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


  0%|          | 0/1 [00:00<?, ?it/s]

## UDT Initialization

We can create a UDT model specific for query reformulation by specifying the name of the source column (column containing queries to be reformulated) and the name of the target column (correct reformulations) and a dataset size parameter. The size of the input dataset can be configured to be either "small" (size < 1M), "medium"(size < 10M) or "large" (size >= 10M). We configure different model parameters depending on the size of the input dataset. 

In [4]:
from thirdai import bolt

model = bolt.UniversalDeepTransformer(
    source_column="source_queries", target_column="target_queries", dataset_size="medium"
)

## Training

We can now train our model in just one line of code. You just have to specify the path to the training file. 

In [5]:
model.train(filename=train_filename)

Loading vectors from 'train_file.csv'
Loaded 630000 vectors from 'train_file.csv' in 2 seconds.


## Evaluation 

Evaluating the UDT model is also just one line of code. Since this UDT model is specific for query reformulation, you need to provide the number of suggested candidate queries that the UDT model generates. For instance, if you want to see the top 10 suggested query reformulations of the input query, set the top_k parameter to 10. Evaluating this model will also print out recall @k. 

In [ ]:
query_reformulations = model.evaluate(filename=test_filename, top_k=5)

## Saving and Loading

Saving and loading a trained UDT model to disk is also extremely straight forward. 

In [ ]:
model_location = "query_reformulation.model"

# Saving
model.save(filename=model_location)

# Loading
model = bolt.UniversalDeepTransformer.load(model_location)

## Testing Predictions 

The evaluation method is great for testing, but it requires labels, which don't exist in a production environment. We also provide a predict method that can take a list of queries or a single query, which allows for easy integration into production pipelines. 

In [ ]:

predictions = model.predict_batch(queries=inference_batch, top_k=5)



In [ ]:
# Remove the created training and test file
import os

os.remove(train_filename)
os.remove(test_filename)